In [18]:
import sys
import xml.etree.ElementTree as ET


sys.path.append('../')
sys.path.append('../my_libs/')
sys.path.append('../my_libs/img/')

!{sys.executable} -m pip install --break-system-packages -r ../../pip-dependencies.txt

Defaulting to user installation because normal site-packages is not writeable


In [12]:
import my_libs as ml
from my_libs.tools import *
from my_libs.fft import *
from my_libs.colors import *
from my_libs.img.processing import *
from ultralytics import YOLO


In [13]:
ml.tools.printf("Hello world!")

Hello world!

## Cvičení 13(14)

0)  Použijte knihovnu ultralytics. 
Knihovnu lze nainstalovat jako pip install ultralytics 
1)  Převeďte data do formátu podporovaného ultralyics dle 
https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/  
Všechny anotace k obrázkům jsou v annotations.xml. Ke každému 
obrázku nazev.png v adresáři images je potřeba vytvořit odpovídající 
nazev.txt v adresáři labels. Každý objekt na obrázku je pak jeden 
řádek v txt souboru ve formátu 
class_idx x_center y_center width height 
kde  class_idx je index třídy a ostatní jsou souřadnice ohraničujícího 
obdélníka relativně vůči velikosti obrázku, viz 
https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#22-create-
labels  
2)  Důležitý je rovněž yaml soubor s cestami k datasetu a názvy tříd (v našem 
problému máme pouze jednu třídu „strawberry”), viz  
https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#21-create-
datasetyaml  
Data nemusíte rozdělovat na trénovací a validační, tzn. pro adresáře 
train i val použijte adresář obsahující všechny obrázky. 
3)  Stáhněte předtrénovaný YOLO model z 
https://docs.ultralytics.com/models/yolo11/#supported-tasks-and-modes 
dostatečně dobře bude fungovat i nejmenší model nano (n). 
4)  Natrénujte model a uložte, viz 
https://docs.ultralytics.com/modes/train/#usage-examples  
5)  Použijte model na obrázky v podadresáři test a vykreslete predikované 
obdélníky, viz 
https://docs.ultralytics.com/modes/predict/#key-features-of-predict-mode

1. Ke každému obrázku má být .txt s třídou (0 protože strawberry) a relativní souřadnice (0-1) středu obrázku a výšky a šířky obdélníka
2. Natrénovat na celém datasetu, nerozdělovat ho na train a test
4. Vykreslit results sám, ne pomocí zabudovaných funkcí

In [23]:
# Load a COCO-pretrained YOLO11n model
model = YOLO("yolo11n.pt", "dataset.yaml")

### Resize obrázky na 640x640px

In [17]:
input_folder = "./strawberries/images"
output_folder = "./strawberries/images_640px"

for file in os.listdir(input_folder):
    input_path = os.path.join(input_folder, file)
    output_path = os.path.join(output_folder, file)

    image = cv2.imread(input_path)
    image_640px = cv2.resize(image, (640, 640))

    cv2.imwrite(output_path, image_640px)
print("Images resized and saved to: ", output_folder)

Images resized and saved to:  ./strawberries/images_640px


### Tvorba labels z annotations.xml

In [29]:
labels_folder = "./strawberries/labels"

def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    annotations = []
    for image in root.findall("image"):
            txt_filename = f"{labels_folder}/{image.attrib["id"]}.txt"
            txt_file = open(txt_filename, "a")
            image_width = int(image.attrib["width"])
            image_height = int(image.attrib["height"])
            for box in range(len(image)):
                x_center = float(image[box].attrib["xtl"])/image_width
                y_center = float(image[box].attrib["ytl"])/image_height
                width = float(image[box].attrib["xbr"])/image_width
                height = float(image[box].attrib["ybr"])/image_height
                txt_file.write(f"0 {x_center} {y_center} {width} {height}\n")

parse_annotation("./strawberries/annotations.xml")